# Test Valuation Notebook

This notebook exercises **all ValuationEngine** classes:
1. `ValuationEngineProductBulletCashflow`
2. `ValuationEngineProductFuture`
3. `ValuationEngineProductRfrFuture`
4. `ValuationEngineProductIborSwap`
5. `ValuationEngineProductOvernightSwap`

We will:
- Build the same dummy `YieldCurve` from test_model
- Instantiate each product
- Price it via its ValuationEngine


In [1]:
import pandas as pd

from product.linear_products import (
    ProductBulletCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
)
from yield_curve.yield_curve_model import YieldCurve
from yield_curve.valuation_engine_yc import (
    ValuationEngineProductBulletCashflow,
    ValuationEngineProductFuture,
    ValuationEngineProductRfrFuture,
    ValuationEngineProductIborSwap,
    ValuationEngineProductOvernightSwap
)


In [2]:
#YieldCurve
DUMMY_DATA = [
    ['SOFR-1B',           '1M',  0.0300],
    ['SOFR-1B',           '3M',  0.0325],
    ['SOFR-1B',           '6M',  0.0350],
    ['SOFR-1B',           '1Y',  0.0375],
    ['USD-LIBOR-BBA-3M',  '1M',  0.0400],
    ['USD-LIBOR-BBA-3M',  '3M',  0.0425],
    ['USD-LIBOR-BBA-3M',  '6M',  0.0450],
    ['USD-LIBOR-BBA-3M',  '1Y',  0.0475],
]
DUMMY_BM = [
    {'TARGET': 'SOFR-1B',          'INTERPOLATION METHOD': 'PIECEWISE_CONSTANT'},
    {'TARGET': 'USD-LIBOR-BBA-3M', 'INTERPOLATION METHOD': 'PIECEWISE_CONSTANT'},
]

data_df = pd.DataFrame(DUMMY_DATA, columns=['INDEX', 'AXIS1', 'VALUES'])
build_methods = DUMMY_BM
model = YieldCurve('2025-05-25', data_df, build_methods)
print("YieldCurve ready.\n")


YieldCurve ready.



In [3]:
# ValuationEngineProductBulletCashflow
bullet_cf = ProductBulletCashflow(
    '2028-05-25',  # terminationDate
    'USD',         # currency
    1e6,           # notional
    'Long'         # longOrShort
)
val_params_bullet = {'FUNDING INDEX': 'SOFR-1B'}
vb = ValuationEngineProductBulletCashflow(model, val_params_bullet, bullet_cf)
vb.calculateValue()
print("Bullet Cashflow PV →", vb.value)


Bullet Cashflow PV → ['USD', np.float64(895460.9488260685)]


In [4]:
# ValuationEngineProductFuture (IBOR future)
fut = ProductFuture(
    effectiveDate='2025-06-01',
    index='USD-LIBOR-BBA-3M',
    strike=98.50,
    notional=5e5,
    longOrShort='Short'
)
vf = ValuationEngineProductFuture(model, {}, fut)
vf.calculateValue()
print("IBOR Future P&L →", vf.value)


IBOR Future P&L → ['USD', np.float64(48770637.97967764)]


In [5]:
# ValuationEngineProductRfrFuture (OIS future)
rfr_fut = ProductRfrFuture(
    effectiveDate='2025-06-01',
    index='SOFR-1B',
    strike=99.75,
    notional=5e5,
    longOrShort='Long'
)
vr = ValuationEngineProductRfrFuture(model, {}, rfr_fut)
vr.calculateValue()
print("OIS Future P&L →", vr.value)

OIS Future P&L → ['USD', np.float64(nan)]


c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib\yield_curve\yield_curve_model.py:77: RuntimeWarning: invalid value encountered in scalar divide
  return (dfStart / dfEnd - 1.) / accrued


In [6]:
# ValuationEngineProductIborSwap (vanilla IBOR swap)
val_params_ibor_swap = {
    'FIXED FREQUENCY': '6M',
    'HOL CONV':       'NYC',
    'BIZ CONV':       'MF',
    'ACC BASIS':      'ACT/360'
}
ibor_swap = ProductIborSwap(
    effectiveDate='2025-06-01',
    maturity='1Y',
    payFixed=True,
    fixedRate=0.0450,
    floatingIndex='USD-LIBOR-BBA-3M',
    notional=2e6,
    longOrShort='Long'
)
vis = ValuationEngineProductIborSwap(model, val_params_ibor_swap, ibor_swap)
vis.calculateValue()
print("IBOR Swap PV →", vis.value)


IBOR Swap PV → ['USD', np.float64(2365.2037001907593)]


In [7]:
# ValuationEngineProductOvernightSwap (vanilla OIS swap)
val_params_ois_swap = {
    'FIXED FREQUENCY': '1Y',
    'HOL CONV':       'TARGET',
    'BIZ CONV':       'MF',
    'ACC BASIS':      '30/360'
}
ois_swap = ProductOvernightSwap(
    effectiveDate='2025-06-01',
    maturity='1Y',
    payFixed=False,
    fixedRate=0.0150,
    overnightIndex='SOFR-1B',
    notional=1e6,
    longOrShort='Short'
)
vos = ValuationEngineProductOvernightSwap(model, val_params_ois_swap, ois_swap)
vos.calculateValue()
print("OIS Swap PV →", vos.value)


OIS Swap PV → ['USD', np.float64(-21011.280852213244)]
